In [ ]:
import numpy as np
import requests
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models


## Question 1

In [5]:
model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

c:\Users\kerim\github_repo\llm-zoomcamp\homework\2025\02-vector-search\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kerim\AppData\Local\Temp\fastembed_cache\models--xenova--jina-embeddings-v2-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

2025-06-24 15:24:20.442 | ERROR    | fastembed.common.model_management:download_model:430 - Could not download model from HuggingFace: [WinError 1314] A required privilege is not held by the client: '..\\..\\blobs\\a8b3208c2884c4efb86e49300fdd3dc877220cdf' -> 'C:\\Users\\kerim\\AppData\\Local\\Temp\\fastembed_cache\\models--xenova--jina-embeddings-v2-small-en\\snapshots\\523cadcb9c2e71c7153fc46016e1fe79acb4f58f\\special_tokens_map.json' Falling back to other sources.
2025-06-24 15:24:20.447 | ERROR    | fastembed.common.model_management:download_model:452 - Could not download model from either source, sleeping for 3.0 seconds, 2 retries left.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
query = "I just discovered the course. Can I join now?"

In [25]:
query_embedding = np.array(next(model.embed([query])))

print(f"The minimal value: {np.min(query_embedding):.2f}")

The minimal value: -0.12


## Question 2

In [21]:
doc = "Can I still join the course after the start date?"

In [22]:
doc_embedding = np.array(next(model.embed([doc])))

In [ ]:
cosine_similarity = np.dot(query_embedding, doc_embedding)

print(
    f"The cosine similarity between the vector for the query and the vector for the document is {cosine_similarity:.1f}"
)

The cosine similarity between the vector for the query and the vector for the document is 0.9


## Question 3

In [28]:
documents = [
    {
        "text": "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
        "section": "General course-related questions",
        "question": "Course - Can I still join the course after the start date?",
        "course": "data-engineering-zoomcamp",
    },
    {
        "text": "Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.",
        "section": "General course-related questions",
        "question": "Course - Can I follow the course after it finishes?",
        "course": "data-engineering-zoomcamp",
    },
    {
        "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
        "section": "General course-related questions",
        "question": "Course - When will the course start?",
        "course": "data-engineering-zoomcamp",
    },
    {
        "text": "You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.",
        "section": "General course-related questions",
        "question": "Course - What can I do before the course starts?",
        "course": "data-engineering-zoomcamp",
    },
    {
        "text": "Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.",
        "section": "General course-related questions",
        "question": "How can we contribute to the course?",
        "course": "data-engineering-zoomcamp",
    },
]

In [ ]:
documents_embed = np.array(
    [np.array(next(model.embed([doc["text"]]))) for doc in documents]
)

In [ ]:
max_index = np.argmax(documents_embed.dot(query_embedding))

In [38]:
print(f"The document index with the highest similarity is {max_index}")

The document index with the highest similarity is 1


## Question 4

In [ ]:
documents_embed = np.array(
    [
        np.array(next(model.embed([doc["question"] + " " + doc["text"]])))
        for doc in documents
    ]
)

In [43]:
max_index = np.argmax(documents_embed.dot(query_embedding))

In [44]:
print(f"The document index with the highest similarity is {max_index}")

The document index with the highest similarity is 0


## Question 5

In [ ]:
MIN_EMBEDDING_DIMENSIONALITY = min(
    [model["dim"] for model in TextEmbedding.list_supported_models()]
)

In [51]:
print(MIN_EMBEDDING_DIMENSIONALITY)

384


## Question 6

In [ ]:
model_handle = "BAAI/bge-small-en"

In [ ]:
docs_url = "https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json"
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course["course"]
    if course_name != "machine-learning-zoomcamp":
        continue

    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)

In [67]:
client = QdrantClient("http://localhost:6333")

In [ ]:
collection_name = "zoomcamp-rag"

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=MIN_EMBEDDING_DIMENSIONALITY, distance=models.Distance.COSINE
    ),
)

True

In [ ]:
points = []

for i, doc in enumerate(documents):
    text = doc["question"] + " " + doc["text"]
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(id=i, vector=vector, payload=doc)
    points.append(point)

In [ ]:
client.upsert(collection_name=collection_name, points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [75]:
question = "I just discovered the course. Can I join now?"

In [ ]:
query_points = client.query_points(
    collection_name=collection_name,
    query=models.Document(text=question, model=model_handle),
    limit=5,
    with_payload=True,
)

In [80]:
query_points

QueryResponse(points=[ScoredPoint(id=14, version=0, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=6, version=0, score=0.86918855, payload={'text': 'Approximately 4 months, but may take more if you want to do some extra activities (an extra project, an article, etc)', 'section': 'General course-related questions', 'question': 'How long is the course?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=